https://developers.google.com/gmail/api/quickstart/python

note: the script below must be run as a .py file in order to launch your browser and authenticate

In [1]:
!hostname

6f492426f82a


In [2]:
!pip install --upgrade google-api-python-client

    100% |████████████████████████████████| 61kB 229kB/s ta 0:00:01
    100% |████████████████████████████████| 225kB 301kB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 434kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 994kB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 920kB/s ta 0:00:01
  Running setup.py bdist_wheel for httplib2 ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/6d/41/4b/2b369d6e2b7eaebcdd423516d3fb659c7658c16a2be8fd04ec
Successfully built httplib2


In [3]:
!pip install --upgrade oauth2client

    100% |████████████████████████████████| 102kB 694kB/s a 0:00:01


In [4]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

In [7]:
!python authorize_send_gmail.py  --noauth_local_webserver

/opt/conda/lib/python3.6/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access token.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=974547311126-ocaee49k8edhe52atntd5nrh2jnapta6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fmail.google.com%2F&access_type=offline&response_type=code

Enter verification code: ^C
Traceback (most recent call last):
  File "authorize_send_gmail.py", line 42, in <module>
    main()
  File "authorize_send_gmail.py", line 27, in main
    creds = tools.run_flow(flow, store)
  File "/opt/conda/lib/python3.6/site-packages/oauth2client/_helpers.py", line 133, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/oauth2client/tools.py", line 240, in run_flow
    code = input('Enter verification code: ').strip()


https://developers.google.com/gmail/api/guides/sending

message text needs to be base64 encoded

In [8]:
import base64
base64.b64encode(bytes('your string', 'utf-8'))

b'eW91ciBzdHJpbmc='

In [9]:
#https://docs.python.org/3/library/email.mime.html
from email.mime.text import MIMEText
from email import encoders
    
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    
    # https://docs.python.org/3/library/base64.html
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}
    # https://stackoverflow.com/questions/606191/convert-bytes-to-a-string
    
    #return {'raw': base64.urlsafe_b64encode(bytes("None",'utf-8'))}
    
    # https://docs.python.org/3.4/library/email-examples.html
    #return {'raw': encoders.encode_base64(message.as_string())} 

In [10]:
msg=create_message("benpayne@umbc.edu", 
                   "ben.is.located@gmail.com", 
                   "a subjct", 
                   "msg text from Python")
msg

{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1cy1hc2NpaSIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogN2JpdAp0bzogYmVuLmlzLmxvY2F0ZWRAZ21haWwuY29tCmZyb206IGJlbnBheW5lQHVtYmMuZWR1CnN1YmplY3Q6IGEgc3ViamN0Cgptc2cgdGV4dCBmcm9tIFB5dGhvbg=='}

In [11]:
from googleapiclient import errors

def send_message(service, user_id, message):
    """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print ('Message Id: %s' % message['id'])
        return message
    # https://stackoverflow.com/questions/42541857/error-using-gmail-api-tuto-using-python-3-except-errors-httperror-error
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)

In [12]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

In [13]:
send_message(service, "me", msg)

Message Id: 167152bd457ceaf8


{'id': '167152bd457ceaf8',
 'threadId': '167152bd457ceaf8',
 'labelIds': ['UNREAD', 'SENT', 'INBOX']}

The following also works